======================================================================================================================================================================================================================================
# <center>Creating a Value-Buy Smartphone: Insights from Leading Brand</center>


### <center>Name  : Catherine Helenna Mulyadi</center>

======================================================================================================================================================================================================================================

# 1. Context

### a. Goal:
Create a new smartphone product with price range from 1 to 5 million IDR for catering market needs in two quarters. This objective will be achieved by finding common smartphone features found in the products and investigating which smartphone brands take lead on market.

### b. Background

A lot of Indonesian citizens are tech-lovers; they just cannot resist the temptation of changing their smartphones periodically when a new version has been released. Some considerations to get a new smartphone include design, camera, and specs. With that being said, current smartphones are increasing in variety, fulfilling market demands based on affordability. As the saying goes, there is a price for a value, customers still would like to purchase a value-buy smartphone so it would not bring too much financial problems.

Your company, X, would like to innovate a new smartphone product with a reasonable quality and price so it can label the product as "value buy" for the next release campaign. Your company has been producing smartphones for economic level (price range from 0.5 to 1 million IDR) and wondering to set up a slightly higher price to gain more revenue with its brand new product.

As a data analyst working in company X, you should dig more information about other smartphone brands so you can help your company figuring out what are key features it should include in its new product and how to set up a good price.<br>

### c. How to Reach the Goal?

1. Get at least 60 data about smartphone sales from web scraping on Tokopedia, is there any outlier values on each column in the dataset? Keep the clean data and please find the central tendency, dispersion, skewness, and kurtosis.

2. Which smartphone brands with price range 1 to 5 million IDR dominates the market? (This is depending on how many products are sold per brand, take top 2 brands)

3. Which city has the highest number of sales? Remember to convert the region name that contains the same word as same name. For example: Jakarta Timur, Jakarta Utara -> Jakarta. See if market competitiveness in a city can affect the number of sales. Key: market competition level in a city = number of smartphones sold / number of stores

4. How is the relationship between product price, product rating, and amount of product sold?

5. Make a word cloud from product name section to see which smartphone feature often pops up. Be sure to remove the brand names first so the word cloud will contain other key words.

6. What is the minimum and maximum price the company can set on their product? Hint: use confidence interval and select the suitable method based on data distribution.


## 2. Data Collection
`method`: Web Scraping  

`tool`: BeautifulSoup, Selenium

### a. Import Libraries

In [1]:
# import data analysis-related libraries
import pandas as pd
import numpy as np

# import web scraping libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests
from bs4 import BeautifulSoup

# for data cleaning task
# from dotenv import load_dotenv
# import os
# from openai import OpenAI

In [2]:
# Load environment variables from .env file
# load_dotenv()

# Initialize the OpenAI client
# client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

### b. Web Scraping

#### i. Scraping General Product Information on Catalogue Page
1: identify the link location for scraping with inspect
<small>
```python
    # Main search URL containing the products
search_url = "link"

driver.get(search_url)

    # Extract product links
soup = BeautifulSoup(driver.page_source, "html.parser")
    # This is from inspect
product_elements = soup.find_all('a', class_='oQ94Awb6LlTiGByQZo8Lyw==')  
    # Adjust according to the correct class for product links
product_links = [elem['href'] for elem in product_elements if 'href' in elem.attrs]
print(product_links)
```
</small>
 2: compare the search link for different pages, so we can scrape product links from different pages.  

 3: find the bigger class that contains all information of each product catalogue.   
 
 4: get other information of the product in that bigger class. 

**Collected data**: link, product_name, product_price, rating, total_sold, store_name, and store_location

#### ii. Scraping Product Details Data on Each Product Link
1:  It's important to use header and requests to get the information. If you use web driver, access will be denied when too many times running the code.
<small>
```python
    # Set up headers to mimic a web browser request
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    # Use requests to fetch the page content
    response = requests.get(url, headers=headers)
    
    # Check if the request was successful
    if response.status_code == 200:
```
</small>

**Collected data**: link, product_description, customer_satisfaction, total_rating_info
- product_description gets the whole paragraphs about product description.
- customer_satisfaction represents the percentage of reviewers satisfied with the product.
- total_rating_info shows the number of giving rating and review.

#### I. Function to give more time to scrape

In [2]:
# Intiate first session driver to do webscrap
driver = webdriver.Safari()

In [3]:
# function to elongate time spent on each web by scrolling down
def scroll_to_load_more(driver, wait_time=5):
    """
    Scrolls down to load more items on the page.
    
    :param driver: The Selenium WebDriver instance.
    :param wait_time: The time to wait for new content to load after each scroll.
    """
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to the bottom of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # Wait for new content to load
        time.sleep(wait_time)
        
        # Calculate the new scroll height and compare it with the last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            break  # Exit if no new content has loaded
        last_height = new_height

#### II. Functions to Get General Product Information
- strip_func is to handle missing values and basic cleaning.
- extract_product_info is to get all the general product information.
- data scraped on March 1st, 12:00 PM.

In [4]:
# Function to clean extra whitespaces or fill empty data
def strip_func(my_var):
    final_result = my_var.text.strip() if my_var else "No name available"
    return final_result

In [5]:
# Function to get general product info
def extract_product_info():
    soup = BeautifulSoup(driver.page_source, "html.parser")
    # Find all product containers by the div class that contains product information
    product_containers = soup.find_all('div', class_='css-5wh65g')  # Find all products
    
    # Initialize a list to store product data
    product_data = []

    # Iterate through each container to extract product information
    for container in product_containers:
        product_link_elem = container.find('a', class_='oQ94Awb6LlTiGByQZo8Lyw==')  # Get the anchor tag with the product link
        if product_link_elem and 'href' in product_link_elem.attrs:
            product_link = product_link_elem['href']
            product_name_elem = container.find('span', class_='_0T8-iGxMpV6NEsYEhwkqEg==')  
            product_price_elem = container.find('div', class_='_67d6E1xDKIzw+i2D2L0tjw==')  
            product_rating_elem = container.find('span', class_='_9jWGz3C-GX7Myq-32zWG9w==')
            total_sold_elem = container.find('span', class_='se8WAnkjbVXZNA8mT+Veuw==')
            store_name_elem = container.find('span',class_='T0rpy-LEwYNQifsgB-3SQw==')
            store_location_elem = container.find('span',class_='pC8DMVkBZGW7-egObcWMFQ== flip')
            
            product_name = strip_func(product_name_elem)
            product_price = strip_func(product_price_elem)
            product_rating = strip_func(product_rating_elem)
            total_sold = strip_func(total_sold_elem)
            store_name = strip_func(store_name_elem)
            store_location = strip_func(store_location_elem)
            
            # Store the product data in a dictionary or tuple
            product_data.append({
                "link": product_link,
                "name": product_name,
                "price": product_price,
                "rating":product_rating,
                "total_sold":total_sold,
                "store_name":store_name,
                "store_location":store_location
            })

    return product_data  # Return a list of product dictionaries

#### III. Function to Get Product Description
- scrape_product_details_data processes each url input to get product_description, customer_satisfaction, and total_rating_info.
- data scraped on Mar 4th, 12:00 PM.

In [6]:
# Scraping Product Details Information
def scrape_product_details_data(url):
    # Set up headers to mimic a web browser request
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    # Use requests to fetch the page content
    response = requests.get(url, headers=headers)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Scrape product description
        description_div = soup.find('div', class_='css-16inwn4')
        product_description = description_div.get_text(separator=' ', strip=True) if description_div else "Product description not found."
        
        # Scrape customer satisfaction percentage
        satisfaction_paragraph = soup.find('p', class_='css-61bzch-unf-heading e1qvo2ff8')
        customer_satisfaction = satisfaction_paragraph.get_text(strip=True) if satisfaction_paragraph else "Customer satisfaction information not found."

        # Scrape ratings information
        rating_paragraph = soup.find('p', class_='css-salgbt-unf-heading e1qvo2ff8')
        rating_info = rating_paragraph.get_text(strip=True) if rating_paragraph else "Rating information not found."

        # Return all collected data
        return {
            'link': url,
            'product_description': product_description,
            'customer_satisfaction': customer_satisfaction,
            'total_rating_info': rating_info
        }
    else:
        return f"Error: Unable to access the page (status code: {response.status_code})."

In [7]:
# Scrape product details data
def scrape_product_details_data(driver,url):
    # Send a request to the URL
    driver.get(url)
    # Parse the HTML content
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Scrape product description
    description_div = soup.find('div', class_='css-16inwn4')
    product_description = description_div.get_text(separator=' ', strip=True) if description_div else "Product description not found."
    
    # Scrape customer satisfaction percentage
    satisfaction_paragraph = soup.find('p', class_='css-61bzch-unf-heading e1qvo2ff8')
    customer_satisfaction = satisfaction_paragraph.get_text(strip=True) if satisfaction_paragraph else "Customer satisfaction information not found."

    # Scrape ratings information
    rating_paragraph = soup.find('p', class_='css-salgbt-unf-heading e1qvo2ff8')
    rating_info = rating_paragraph.get_text(strip=True) if rating_paragraph else "Rating information not found."

    # Return all collected data
    return {
        'link':url,
        'product_description': product_description,
        'customer_satisfaction': customer_satisfaction,
        'total_rating_info': rating_info
    }

In [8]:
# # Main search URL containing the products
# search_url = "https://www.tokopedia.com/search?condition=1&fcity=174%2C175%2C176%2C177%2C178%2C179%23144%2C146%2C150%2C151%2C167%2C168%2C171%2C174%2C175%2C176%2C177%2C178%2C179%2C463%23165%2346%23252&navsource=home&pmax=5000000&pmin=1000000&search_id=20250301011425C73A1F58F08846254N3B&srp_component_id=04.06.00.00&srp_page_id=&srp_page_title=&st=product&q=hp%20murah"
# search_url2 = "https://www.tokopedia.com/search?condition=1&fcity=174%2C175%2C176%2C177%2C178%2C179%23144%2C146%2C150%2C151%2C167%2C168%2C171%2C174%2C175%2C176%2C177%2C178%2C179%2C463%23165%2346%23252&navsource=&page={page}&pmax=5000000&pmin=1000000&q=hp%20murah&sc=24&search_id=202503010158437659E76192BAE0069WUB&srp_component_id=04.06.00.00&srp_page_id=&srp_page_title=&st="

# # Collect product links
# all_product_info = []

# # Loop through pages to scrape
# for page in range(1, 10):  # Change range as needed to scrape more pages
#     print(f"Scraping page {page}...")

#     if page == 1:
#         driver.get(search_url)
        
#     else:
#         driver.get(search_url2.format(page=page))  # Format the URL with the current page number

#     # Call the scrolling function to load more items
#     scroll_to_load_more(driver)
#     # time.sleep(2)  # Wait to ensure elements are fully loaded
    
#     # Extract product links
#     current_links = extract_product_info()
#     all_product_info.extend(current_links)
#     print(f"Total links gathered from page {page}: {len(current_links)}")

# # Close the driver when done
# driver.quit()

# # Output the total number of collected product links
# print(f"Total links collected: {len(all_product_info)}")

In [9]:
# # Initiate session again
# driver = webdriver.Safari()
# # Second session
# search_url2 = "https://www.tokopedia.com/search?condition=1&fcity=174%2C175%2C176%2C177%2C178%2C179%23144%2C146%2C150%2C151%2C167%2C168%2C171%2C174%2C175%2C176%2C177%2C178%2C179%2C463%23165%2346%23252&navsource=&page={page}&pmax=5000000&pmin=1000000&q=hp%20murah&sc=24&search_id=202503010158437659E76192BAE0069WUB&srp_component_id=04.06.00.00&srp_page_id=&srp_page_title=&st="

# # Loop through pages to scrape
# for page in range(11, 20):  # Change range as needed to scrape more pages
#     print(f"Scraping page {page}...")
#     driver.get(search_url2.format(page=page))  # Format the URL with the current page number

#     # Call the scrolling function to load more items
#     scroll_to_load_more(driver)
#     # time.sleep(2)  # Wait to ensure elements are fully loaded
    
#     # Extract product links
#     current_links = extract_product_info()
#     all_product_info.extend(current_links)
#     print(f"Total links gathered from page {page}: {len(current_links)}")

# # Close the driver when done
# driver.quit()

# # Output the total number of collected product links
# print(f"Total links collected: {len(all_product_info)}")

In [10]:
# # Initiate session again
# driver = webdriver.Safari()
# # Second session
# search_url2 = "https://www.tokopedia.com/search?condition=1&fcity=174%2C175%2C176%2C177%2C178%2C179%23144%2C146%2C150%2C151%2C167%2C168%2C171%2C174%2C175%2C176%2C177%2C178%2C179%2C463%23165%2346%23252&navsource=&page={page}&pmax=5000000&pmin=1000000&q=hp%20murah&sc=24&search_id=202503010158437659E76192BAE0069WUB&srp_component_id=04.06.00.00&srp_page_id=&srp_page_title=&st="

# # Loop through pages to scrape
# for page in range(21, 34):  # Change range as needed to scrape more pages
#     print(f"Scraping page {page}...")
#     driver.get(search_url2.format(page=page))  # Format the URL with the current page number

#     # Call the scrolling function to load more items
#     scroll_to_load_more(driver)
#     # time.sleep(2)  # Wait to ensure elements are fully loaded
    
#     # Extract product links
#     current_links = extract_product_info()
#     all_product_info.extend(current_links)
#     print(f"Total links gathered from page {page}: {len(current_links)}")

# # Close the driver when done
# driver.quit()

# # Output the total number of collected product links
# print(f"Total links collected: {len(all_product_info)}")

In [11]:
# # Initiate session again
# driver = webdriver.Safari()
# # Failed to scrape pages
# list_of_pages1 = [2, 3, 5, 6, 8]
# list_of_pages2 = [13, 15, 16, 17, 18, 19]
# list_of_pages3 = [22, 23, 25, 27, 29, 32, 33]

# # Combine all the page numbers into one list
# all_failed_pages = list_of_pages1 + list_of_pages2 + list_of_pages3

# # Second session
# search_url2 = "https://www.tokopedia.com/search?condition=1&fcity=174%2C175%2C176%2C177%2C178%2C179%23144%2C146%2C150%2C151%2C167%2C168%2C171%2C174%2C175%2C176%2C177%2C178%2C179%2C463%23165%2346%23252&navsource=&page={page}&pmax=5000000&pmin=1000000&q=hp%20murah&sc=24&search_id=202503010158437659E76192BAE0069WUB&srp_component_id=04.06.00.00&srp_page_id=&srp_page_title=&st="

# # Loop through specific failed pages to scrape
# for page in all_failed_pages: 
#     print(f"Scraping page {page}...")
#     driver.get(search_url2.format(page=page))  # Format the URL with the current page number

#     # Call the scrolling function to load more items
#     scroll_to_load_more(driver)

#     # Extract product links
#     current_links = extract_product_info()  # Ensure this function correctly extracts product links
#     all_product_info.extend(current_links)
#     print(f"Total links gathered from page {page}: {len(current_links)}")

# # Close the driver when done
# driver.quit()

# # Output the total number of collected product links
# print(f"Final total links collected: {len(all_product_info)}")

In [12]:
# Data Saving
# data_info=pd.DataFrame(all_product_info)
# data_info.to_csv("general_product_info.csv")

In [13]:
# # Load the data
# data_info=pd.read_csv("general_product_info.csv")
# data_info.head()

In [14]:
# # Create an empty list to store product details
# product_details_list = []  
# try:
#     # Proceed with scraping product details from the links
#     for link in data_info['link']:
#         product_details_dict = scrape_product_details_data(link)  # Call the function without the driver
#         product_details_list.append(product_details_dict)  # Append the details to the list

# except Exception as e:
#     print(f"An error occurred: {e}")

# # Output the list of product details
# print(product_details_list)

In [15]:
# create a separate dataframe
# product_details_df = pd.DataFrame(product_details_list)
# print("Data Layout:")
# print(product_details_df.head())
# print("\n Show product details:")
# print(product_details_df.info())

In [16]:
# Data Saving
# product_details_df.to_csv("product_details_data.csv")

## 3. Data Preparation
- 1: Reload data from scraped sources. Note missing values are tagged as "No name available"
- 2: Data Profiling and Transformation. Identify data types, clean data, transform by extracting information or remove irrelevant data.
- 3: Perform EDA to understand the basic data profile.  
- 4: Transform and clean data for analytical purposes.

### i. Data Loading and Basic Profiling

In [20]:
# load data from csv file
raw_gen_prod_info = pd.read_csv("general_product_info.csv")

# check first 5 data
raw_gen_prod_info.head()

,Unnamed: 0,link,name,price,rating,total_sold,store_name,store_location
0,0,https://www.tokopedia.com/waspa10/new-arrival-...,New arrival SmartPhone Pova 6 Pro 7.3 inch Lay...,Rp1.045.000,5.0,2 terjual,waspa10 store,Surabaya
1,1,https://www.tokopedia.com/mafiakaos/hp-vivo-y2...,HP VIVO Y27 RAM 8 GB 256 GB 5G NFC BARU MURAH,Rp1.200.000,3.8,7 terjual,Mafia Kaos,Jakarta Barat
2,2,https://www.tokopedia.com/queenofgadget/hp-xla...,HP XlAOMl REDMl 12 8/256 GB FullSet New Handph...,Rp1.119.000,4.1,22 terjual,Queen of Gadget,Jakarta Barat
3,3,https://www.tokopedia.com/kalolakuaja/itel-rs4...,ITEL RS4 12gb/256gb 12/256 8/128 Baru new Ori...,Rp2.099.000,5.0,No name available,KALOLAKUAJA,Jakarta Selatan
4,4,https://www.tokopedia.com/archive-tria3434/iph...,iPhone XS MAX Ultimate Dual SIM | Hp Batam Har...,Rp1.999.000,No name available,No name available,tria3434,Medan


In [22]:
# check data type
raw_gen_prod_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 393 entries, 0 to 392
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      393 non-null    int64 
 1   link            393 non-null    object
 2   name            393 non-null    object
 3   price           393 non-null    object
 4   rating          393 non-null    object
 5   total_sold      393 non-null    object
 6   store_name      393 non-null    object
 7   store_location  393 non-null    object
dtypes: int64(1), object(7)
memory usage: 24.7+ KB


In [21]:
# load data from csv file
raw_prod_details = pd.read_csv("product_details_data.csv")

# check first 5 data
raw_prod_details.head()

,Unnamed: 0,link,product_description,customer_satisfaction,total_rating_info
0,0,https://www.tokopedia.com/waspa10/new-arrival-...,"Penjualan langsung pabrik, kualitas yang lebih...",100% pembeli merasa puas,1 rating • 0 ulasan
1,1,https://www.tokopedia.com/mafiakaos/hp-vivo-y2...,READY STOK SIAP KIRIM WARNA HITAM DAN BIRU BAR...,60% pembeli merasa puas,5 rating • 4 ulasan
2,2,https://www.tokopedia.com/queenofgadget/hp-xla...,Welcome to QueenGadget ! -Diharapkan untuk mem...,81% pembeli merasa puas,11 rating • 4 ulasan
3,3,https://www.tokopedia.com/kalolakuaja/itel-rs4...,"Cicilan Mudah dengan atau Tanpa Kartu kredit, ...",100% pembeli merasa puas,1 rating • 1 ulasan
4,4,https://www.tokopedia.com/archive-tria3434/iph...,"Ready Stok ya kak : Rosegold, White, Black IPH...",Customer satisfaction information not found.,Rating information not found.


In [23]:
# check data type
raw_prod_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 393 entries, 0 to 392
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Unnamed: 0             393 non-null    int64 
 1   link                   393 non-null    object
 2   product_description    390 non-null    object
 3   customer_satisfaction  393 non-null    object
 4   total_rating_info      393 non-null    object
dtypes: int64(1), object(4)
memory usage: 15.5+ KB


#### **Insight:**
- There are total of 393 data with some missing values tagged with "No name available","Customer satisfaction information not found." (customer_satisfaction column),"Rating information not found" (total_rating_info column), and NaN (product_description column in raw_prod_details dataframe.)
- The index columns from datasets are stored under Unnamed:0; they need to be removed.
- Some columns have incorrect data types: price, rating, total_sold, customer_satisfaction, total_rating_info. These should be numerical type.
- The link column acts as the foreign key to connect both columns.

### ii. Data Cleaning for Each Column

#### a. Cleaning Function Collection

In [75]:
def drop_first_column(df):
    """
    Drop the first column of the DataFrame, specifically targeting a column named "Unnamed: 0".
    If the column does not exist, the DataFrame is returned unchanged.
    """
    if "Unnamed: 0" in df.columns:
        df = df.drop(columns=["Unnamed: 0"], axis=1)
    return df

1. **​rating**:
- Convert the data type to float.
- Fill unknown values with a neutral rating of 2.5 and flag them with IsRated = False.

In [76]:
# process rating
def process_ratings(df, rating_column='Rating', neutral_value=2.5, missing_value='No name available'):
    """
    Processes a DataFrame with a 'Rating' column by:
    1. Replacing 'No name available' with NaN.
    2. Adding a new column 'IsRated' to indicate whether the product has been rated.
    3. Filling NaN values with a neutral placeholder (default: 2.5).
    4. Converting the column to float.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        rating_column (str): The name of the column containing ratings (default: 'Rating').
        neutral_value (float): The neutral value to fill for unrated products (default: 2.5).
        missing_value (str): The string representing missing values (default: 'No name available').

    Returns:
        pd.DataFrame: The processed DataFrame with filled ratings and the 'IsRated' column.
    """
    # Step 1: Replace 'No name available' with NaN
    df[rating_column] = df[rating_column].replace(missing_value, np.nan)
    
    # Step 2: Add a new column 'IsRated' to indicate whether the product has been rated
    df['IsRated'] = df[rating_column].notna()  # True if rated, False if unrated
    
    # Step 3: Convert the column to float
    df[rating_column] = df[rating_column].astype(float)
    
    # Step 4: Fill NaN values with the neutral placeholder
    df[rating_column] = df[rating_column].fillna(neutral_value)
    
    return df

2. **​price**:
- Remove the string "Rp.".
- Replace "No name available" with 0.
- Convert the data type to float. Use 0 as a flag for unknown phone prices.

In [86]:
# process price
def process_price_column(df, price_column='Price'):
    """
    Processes a DataFrame with a 'Price' column by:
    1. Removing the 'Rp.' prefix.
    2. Replacing 'No name Available' with 0.
    3. Converting the column to the float data type.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        price_column (str): The name of the column containing prices (default: 'Price').

    Returns:
        pd.DataFrame: The processed DataFrame with the updated 'Price' column.
    """
    # Step 1: Remove 'Rp.' prefix and the dot (.) in one step
    df[price_column] = df[price_column].str.replace('Rp\.', '', regex=True)
    df[price_column] = df[price_column].replace('Rp',"",regex = True)
    # Step 2: Use backslash symbol before '.' to ensure the removal is the string value of "."
    df[price_column] = df[price_column].replace('\.',"", regex = True)
    
    # Step 2: Replace 'No name Available' with 0
    df[price_column] = df[price_column].replace('No name Available', '0')
    
    # Step 3: Convert the column to float
    df[price_column] = df[price_column].astype(float)
    
    return df

3. ​**total_sold**:
- Remove unnecessary strings ('+', 'terjual', and spaces).
- Convert "rb" to "000" (e.g., "1 rb" → 1000).
- Replace 'No name Available' with 0.
- Convert the data type to integer.

In [94]:
def clean_total_sold(df, column='Total Sold'):
    """
    Cleans the 'Total Sold' column by:
    1. Removing unnecessary string values ('+', 'terjual', and spaces).
    2. Replacing 'No name Available' with '0'.
    3. Replacing 'rb' with '000' to handle thousands.
    4. Converting the column to the int data type.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        column (str): The name of the column to clean (default: 'Total Sold').

    Returns:
        pd.DataFrame: The DataFrame with the cleaned and converted column.
    """
    # Step 1: Remove unnecessary string values
    df[column] = df[column].replace('\+', '', regex=True)  # Remove '+'
    df[column] = df[column].replace('terjual', '', regex=True)  # Remove 'terjual'
    df[column] = df[column].replace(' ', '', regex=True)  # Remove spaces
    
    # Step 2: Replace 'No name Available' with '0'
    df[column] = df[column].replace('No name Available', '0')  # Replace before spaces are removed
    df[column] = df[column].replace('Nonameavailable', '0')  # Replace after spaces are removed
    
    # Step 3: Replace 'rb' with '000' to handle thousands
    df[column] = df[column].replace('rb', '000', regex=True)
    
    # Step 4: Convert the column to int
    df[column] = df[column].astype(int)
    
    return df


​4. **total_rating_info**:
- for "Rating information not found", give 0. this means, there is no rating.
- Extract the number before "rating" (e.g., "5 rating" → 5).
- Convert the data type to integer.
- make new column called review_rating_ratio.
- Extract the number before "ulasan" to get the number of reviews.
- Divide the number of reviews by total_rating_info to calculate the ratio.
- If total_rating_info or the number of reviews are 0, set the ratio to 0.

In [100]:
# total_rating_info
def clean_rating_info(df):
    """
    Clean the DataFrame by processing the `total_rating_info` column and adding new columns:
    - `total_rating`: Extracted total ratings.
    - `num_reviews`: Extracted number of reviews.
    - `review_rating_ratio`: Ratio of reviews to total ratings.

    Updates:
    - Handle "Rating information not found" by setting total_rating, num_reviews, and review_rating_ratio to 0.
    - Extract the number before "rating" and convert to integer.
    - Extract the number before "ulasan" to get the number of reviews.
    - Calculate the review_rating_ratio as num_reviews / total_rating.
    - If total_rating or num_reviews is 0, set review_rating_ratio to 0.
    """
    # Define a helper function to process each row
    def process_row(row):
        # Check if the row contains "Rating information not found"
        if row['total_rating_info'] == 'Rating information not found':
            return 0, 0, 0
        
        try:
            # Split the string into parts
            parts = row['total_rating_info'].split('•')
            
            # Extract the number before "rating"
            rating_part = parts[0].strip()
            total_rating = int(rating_part.split()[0])  # Extract the first word and convert to integer
            
            # Extract the number before "ulasan"
            review_part = parts[1].strip()
            num_reviews = int(review_part.split()[0])  # Extract the first word and convert to integer
            
            # Calculate the review_rating_ratio
            review_rating_ratio = num_reviews / total_rating if total_rating != 0 else 0
            
            # Return the processed values
            return total_rating, num_reviews, review_rating_ratio
        except (IndexError, ValueError, AttributeError):
            # Handle cases where the format is invalid or missing
            return 0, 0, 0

    # Apply the helper function to each row and create new columns
    df[['total_rating', 'num_reviews', 'review_rating_ratio']] = df.apply(process_row, axis=1, result_type='expand')
    
    # Drop the `total_rating_info` column after all transformations are finished
    df = df.drop(columns=['total_rating_info'])
    return df


5. ​**customer_satisfaction**:
- Extract the numerical part before "%" (e.g., "85%" → 85).
- For 'No name Available', convert to 50 if both total_rating_info and total_sold are 0. This implies no customer feedback or an average purchasing experience.
- Convert the data type to float.

In [98]:
def clean_customer_satisfaction(df, customer_satisfaction_col='customer_satisfaction', 
                                total_rating_info_col='total_rating_info', total_sold_col='total_sold',rating_col='rating'):
    """
    Clean the `customer_satisfaction` column by:
    1. Extracting the numerical part before "%" (e.g., "100% pembeli merasa puas" → 100).
    2. For 'No name Available', convert to 50 if both `total_rating_info` and `total_sold` are 0.
    3. Converting the data type to float.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        customer_satisfaction_col (str): The name of the `customer_satisfaction` column (default: 'customer_satisfaction').
        total_rating_info_col (str): The name of the `total_rating_info` column (default: 'total_rating_info').
        total_sold_col (str): The name of the `total_sold` column (default: 'total_sold').

    Returns:
        pd.DataFrame: The DataFrame with the cleaned `customer_satisfaction` column.
    """
    # Define a helper function to process each row
    def process_row(row):
        # Handle 'No name Available'
        if row[customer_satisfaction_col] == 'No name Available':
            # Check if both `total_rating_info` and `total_sold` are 0
            if row[total_rating_info_col] == 0 and row[total_sold_col] == 0:
                return 50.0  # Implies average customer satisfaction
            elif row[total_rating_info_col] !=0:
                if row[rating_col] == 0:
                    return 50.0
                else:
                    min_rating = 0
                    max_rating = 5
                    max_satisfaction = 100
                    
                    # Ensure the rating is within the valid range
                    if row[rating_col] < min_rating or row[rating_col] > max_rating:
                        raise ValueError(f"Rating must be between {min_rating} and {max_rating}.")
                    
                    # Calculate the satisfaction score
                    satisfaction_score = (row[rating_col] / max_rating) * max_satisfaction
                    return satisfaction_score 
            else:
                return 0.0  # Default value for 'No name Available' if conditions are not met
        
        # Extract the numerical part before "%"
        if isinstance(row[customer_satisfaction_col], str) and '%' in row[customer_satisfaction_col]:
            # Split the string by "%" and take the first part
            numerical_part = row[customer_satisfaction_col].split('%')[0].strip()
            try:
                return float(numerical_part)  # Convert to float
            except ValueError:
                return 0.0  # Default value if conversion fails
        else:
            return 0.0  # Default value for invalid or missing data

    # Apply the helper function to each row and update the column
    df[customer_satisfaction_col] = df.apply(process_row, axis=1)
    
    return df

6. **product_description and store_name**:
- Capitalize data in store_name.
- Small caps on product_description. Remove the missing rows.

In [ ]:
# for store_name and product_description
def letter_transformer(df):
    """
    Clean the DataFrame by:
    1. Capitalizing each word in 'store_name'.
    2. Converting 'product_description' to lowercase.
    3. Dropping rows with missing 'product_description'.
    """
    # Drop rows with missing 'product_description'
    df = df.dropna(subset=['product_description'])
    
    # 1. Capitalize each word in store_name
    df['store_name'] = df['store_name'].str.title()
    
    # 2. Convert product_description to lowercase
    df['product_description'] = df['product_description'].str.lower()
    
    return df

7. **brand_name**:
- see the product_name pattern to generate the brand_name.
- `problem`: there is no clear pattern.
- `solution`: begin with collection of generic brands. then, match with the rows. after that, check on how to extract brand name from individual cases.

In [134]:
# Example usage
def extract_brand_name(product_name):
    # Define the prompt
    prompt = f"""
    Extract the brand name from the following product name. 
    If no brand name is found, return "Unknown".
    Product Name: {product_name}
    Brand Name:
    """
    
    # Call the OpenAI API
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # Use a chat model
        messages=[
            {"role": "system", "content": "You are a brand expert."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=10,  # Limit the response length
        temperature=0,  # Set to 0 for deterministic output
    )
    
    # Extract and clean the brand name
    brand_name = response.choices[0].message.content.strip()
    return brand_name

In [ ]:
# Example usage
product_name = "HP VIVO Y27 RAM 8 GB 256 GB 5G NFC BARU MURAH"
brand_name = extract_brand_name(product_name)
print(brand_name)  # Output: VIVO

Brand Name: VIVO


#### b. Data Merging

In [ ]:
# Drop the first column
raw_gen_prod_info_df = drop_first_column(raw_gen_prod_info)
raw_prod_details_df = drop_first_column(raw_prod_details)

In [69]:
# Check raw_gen_prod_info before and after
print("Shape of raw_gen_prod_info before dropping:", raw_gen_prod_info.shape)
print("First few rows of raw_gen_prod_info before dropping:")
print(raw_gen_prod_info.head())

raw_gen_prod_info_df = drop_first_column(raw_gen_prod_info)
print("Shape of raw_gen_prod_info after dropping:", raw_gen_prod_info_df.shape)
print("First few rows of raw_gen_prod_info after dropping:")
print(raw_gen_prod_info_df.head())

# Check raw_prod_details before and after
print("Shape of raw_prod_details before dropping:", raw_prod_details.shape)
print("First few rows of raw_prod_details before dropping:")
print(raw_prod_details.head())

raw_prod_details_df = drop_first_column(raw_prod_details)
print("Shape of raw_prod_details after dropping:", raw_prod_details_df.shape)
print("First few rows of raw_prod_details after dropping:")
print(raw_prod_details_df.head())

Shape of raw_gen_prod_info before dropping: (393, 8)
First few rows of raw_gen_prod_info before dropping:
   Unnamed: 0                                               link  \
0           0  https://www.tokopedia.com/waspa10/new-arrival-...   
1           1  https://www.tokopedia.com/mafiakaos/hp-vivo-y2...   
2           2  https://www.tokopedia.com/queenofgadget/hp-xla...   
3           3  https://www.tokopedia.com/kalolakuaja/itel-rs4...   
4           4  https://www.tokopedia.com/archive-tria3434/iph...   

                                                name        price  \
0  New arrival SmartPhone Pova 6 Pro 7.3 inch Lay...  Rp1.045.000   
1      HP VIVO Y27 RAM 8 GB 256 GB 5G NFC BARU MURAH  Rp1.200.000   
2  HP XlAOMl REDMl 12 8/256 GB FullSet New Handph...  Rp1.119.000   
3  ITEL RS4  12gb/256gb 12/256 8/128 Baru new Ori...  Rp2.099.000   
4  iPhone XS MAX Ultimate Dual SIM | Hp Batam Har...  Rp1.999.000   

              rating         total_sold       store_name   store_locat

In [71]:
# Remove duplicates from link columns
raw_gen_prod_info_df = raw_gen_prod_info_df.drop_duplicates(subset=['link'])
raw_prod_details_df = raw_prod_details_df.drop_duplicates(subset=['link'])

# Clean key columns (e.g., strip spaces)
raw_gen_prod_info_df['link'] = raw_gen_prod_info_df['link'].str.strip()
raw_prod_details_df['link'] = raw_prod_details_df['link'].str.strip()

# Find links in raw_prod_details_df that are not in raw_gen_prod_info_df
mismatched_links = raw_prod_details_df[~raw_prod_details_df['link'].isin(raw_gen_prod_info_df['link'])]
print("Mismatched Links:")
print(mismatched_links['link'].unique())

# Check for leading/trailing spaces in both DataFrames
print("Links with leading/trailing spaces in raw_gen_prod_info_df:")
print(raw_gen_prod_info_df[raw_gen_prod_info_df['link'] != raw_gen_prod_info_df['link'].str.strip()]['link'].unique())

print("Links with leading/trailing spaces in raw_prod_details_df:")
print(raw_prod_details_df[raw_prod_details_df['link'] != raw_prod_details_df['link'].str.strip()]['link'].unique())

# Check for case sensitivity issues
print("Case-sensitive mismatches:")
print(raw_prod_details_df[~raw_prod_details_df['link'].str.lower().isin(raw_gen_prod_info_df['link'].str.lower())]['link'].unique())


Mismatched Links:
[]
Links with leading/trailing spaces in raw_gen_prod_info_df:
[]
Links with leading/trailing spaces in raw_prod_details_df:
[]
Case-sensitive mismatches:
[]


In [72]:
# Perform the merge
merged_phone_df = pd.merge(raw_gen_prod_info_df, raw_prod_details_df, on='link', how='inner')
print(merged_phone_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259 entries, 0 to 258
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   link                   259 non-null    object
 1   name                   259 non-null    object
 2   price                  259 non-null    object
 3   rating                 259 non-null    object
 4   total_sold             259 non-null    object
 5   store_name             259 non-null    object
 6   store_location         259 non-null    object
 7   product_description    256 non-null    object
 8   customer_satisfaction  259 non-null    object
 9   total_rating_info      259 non-null    object
dtypes: object(10)
memory usage: 20.4+ KB
None


In [73]:
# show data layout
merged_phone_df.head()

,link,name,price,rating,total_sold,store_name,store_location,product_description,customer_satisfaction,total_rating_info
0,https://www.tokopedia.com/waspa10/new-arrival-...,New arrival SmartPhone Pova 6 Pro 7.3 inch Lay...,Rp1.045.000,5.0,2 terjual,waspa10 store,Surabaya,"Penjualan langsung pabrik, kualitas yang lebih...",100% pembeli merasa puas,1 rating • 0 ulasan
1,https://www.tokopedia.com/mafiakaos/hp-vivo-y2...,HP VIVO Y27 RAM 8 GB 256 GB 5G NFC BARU MURAH,Rp1.200.000,3.8,7 terjual,Mafia Kaos,Jakarta Barat,READY STOK SIAP KIRIM WARNA HITAM DAN BIRU BAR...,60% pembeli merasa puas,5 rating • 4 ulasan
2,https://www.tokopedia.com/queenofgadget/hp-xla...,HP XlAOMl REDMl 12 8/256 GB FullSet New Handph...,Rp1.119.000,4.1,22 terjual,Queen of Gadget,Jakarta Barat,Welcome to QueenGadget ! -Diharapkan untuk mem...,81% pembeli merasa puas,11 rating • 4 ulasan
3,https://www.tokopedia.com/kalolakuaja/itel-rs4...,ITEL RS4 12gb/256gb 12/256 8/128 Baru new Ori...,Rp2.099.000,5.0,No name available,KALOLAKUAJA,Jakarta Selatan,"Cicilan Mudah dengan atau Tanpa Kartu kredit, ...",100% pembeli merasa puas,1 rating • 1 ulasan
4,https://www.tokopedia.com/archive-tria3434/iph...,iPhone XS MAX Ultimate Dual SIM | Hp Batam Har...,Rp1.999.000,No name available,No name available,tria3434,Medan,"Ready Stok ya kak : Rosegold, White, Black IPH...",Customer satisfaction information not found.,Rating information not found.


#### c. Data Cleaning Implementation (Part 1)

In [101]:
# Running all the data cleaning functions
cleaned_merged_phone_df = merged_phone_df.copy()

# Step 1: clean and transform rating data
cleaned_merged_phone_df = process_ratings(cleaned_merged_phone_df, rating_column='rating', neutral_value=2.5, missing_value='No name available')
# Step 2: clean and transform price data
cleaned_merged_phone_df = process_price_column(cleaned_merged_phone_df, price_column='price')
# Step 3: clean and transform total_sold data
cleaned_merged_phone_df = clean_total_sold(cleaned_merged_phone_df, column='total_sold')
# Step 4: transform total_rating_info data
cleaned_merged_phone_df = clean_rating_info(cleaned_merged_phone_df)
# Step 5: transform store_name and product_description data
cleaned_merged_phone_df = letter_transformer(cleaned_merged_phone_df)
# Step 6: transform customer_satisfaction data
cleaned_merged_phone_df = clean_customer_satisfaction(cleaned_merged_phone_df, customer_satisfaction_col='customer_satisfaction', 
                                total_rating_info_col='total_rating_info', total_sold_col='total_sold',rating_col='rating')

# show result
cleaned_merged_phone_df.head()

,link,name,price,rating,total_sold,store_name,store_location,product_description,customer_satisfaction,IsRated,total_rating,num_reviews,review_rating_ratio
0,https://www.tokopedia.com/waspa10/new-arrival-...,New arrival SmartPhone Pova 6 Pro 7.3 inch Lay...,1045000.0,5.0,2,Waspa10 Store,Surabaya,"penjualan langsung pabrik, kualitas yang lebih...",100.0,True,1.0,0.0,0.000000
1,https://www.tokopedia.com/mafiakaos/hp-vivo-y2...,HP VIVO Y27 RAM 8 GB 256 GB 5G NFC BARU MURAH,1200000.0,3.8,7,Mafia Kaos,Jakarta Barat,ready stok siap kirim warna hitam dan biru bar...,60.0,True,5.0,4.0,0.800000
2,https://www.tokopedia.com/queenofgadget/hp-xla...,HP XlAOMl REDMl 12 8/256 GB FullSet New Handph...,1119000.0,4.1,22,Queen Of Gadget,Jakarta Barat,welcome to queengadget ! -diharapkan untuk mem...,81.0,True,11.0,4.0,0.363636
3,https://www.tokopedia.com/kalolakuaja/itel-rs4...,ITEL RS4 12gb/256gb 12/256 8/128 Baru new Ori...,2099000.0,5.0,0,Kalolakuaja,Jakarta Selatan,"cicilan mudah dengan atau tanpa kartu kredit, ...",100.0,True,1.0,1.0,1.000000
4,https://www.tokopedia.com/archive-tria3434/iph...,iPhone XS MAX Ultimate Dual SIM | Hp Batam Har...,1999000.0,2.5,0,Tria3434,Medan,"ready stok ya kak : rosegold, white, black iph...",0.0,False,0.0,0.0,0.000000


In [102]:
# show the data type and total missing values
cleaned_merged_phone_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259 entries, 0 to 258
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   link                   259 non-null    object 
 1   name                   259 non-null    object 
 2   price                  259 non-null    float64
 3   rating                 259 non-null    float64
 4   total_sold             259 non-null    int64  
 5   store_name             259 non-null    object 
 6   store_location         259 non-null    object 
 7   product_description    256 non-null    object 
 8   customer_satisfaction  259 non-null    float64
 9   IsRated                259 non-null    bool   
 10  total_rating           259 non-null    float64
 11  num_reviews            259 non-null    float64
 12  review_rating_ratio    259 non-null    float64
dtypes: bool(1), float64(6), int64(1), object(5)
memory usage: 24.7+ KB


#### **Insight:**
- the data is successfully merged and transformed in desired format for further analysis.

#### d. Data Cleaning Implementation (Part 2)
- this is for extracting brand_name from name (product_name)

In [ ]:
# print(cleaned_merged_phone_df.name.unique())

['New arrival SmartPhone Pova 6 Pro 7.3 inch Layar HD 16GB+1TB hp murah cuci gudang 2024 asli ori Dual Card SIM 5G Camera 72MP+108MP HD CFace ID Original Cheap Smartphone Ponsel Asli Baru 100% Brand New'
 'HP VIVO Y27 RAM 8 GB 256 GB 5G NFC BARU MURAH'
 'HP XlAOMl REDMl 12 8/256 GB FullSet New Handphone Smartphone Murah'
 'ITEL RS4  12gb/256gb 12/256 8/128 Baru new Original Resmi HP game murah'
 'iPhone XS MAX Ultimate Dual SIM | Hp Batam Harga Murah | Garansi Pstor'
 'Hp baru Reno9 pro handphone murah Original android terbaru 2023'
 'Hp Terbaru 2023 V27e 5G RAM 12GB ROM 512GB 7.5inci handphone murah hp'
 'Ready 【FLASH SALE】 hp murah Reno9 Pro+ 5G 7.5Inci Ponsel Jaminan'
 'Handphone Murah S22 Ultra 5G layar 6.7inch'
 'Hp VIVO V29 12/128GB New Garansi 1 Tahun | Hp Murah Bagus Fullset'
 'Ponsel Android pintar hp OPP0 Reno9 Pro 4G/5G asli baru 16GB+512GB RAM 7,5 inci 6800mAh komputer murah untuk dipelajari siswa jaminan keaslian resmi 32MP+64MP HD kamera obralan smartphone murah terlaris 

#### **Insight**:
- very tough data extraction for brand_name, there is no clear pattern.
- let's use LLMs

In [ ]:
# apply the llm function to each row of data to generate brand_name
# cleaned_merged_phone_df['brand_name'] = cleaned_merged_phone_df['name'].apply(extract_brand_name)

# cleaned_merged_phone_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259 entries, 0 to 258
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   link                   259 non-null    object 
 1   name                   259 non-null    object 
 2   price                  259 non-null    float64
 3   rating                 259 non-null    float64
 4   total_sold             259 non-null    int64  
 5   store_name             259 non-null    object 
 6   store_location         259 non-null    object 
 7   product_description    256 non-null    object 
 8   customer_satisfaction  259 non-null    float64
 9   IsRated                259 non-null    bool   
 10  total_rating           259 non-null    float64
 11  num_reviews            259 non-null    float64
 12  review_rating_ratio    259 non-null    float64
 13  brand_name             259 non-null    object 
dtypes: bool(1), float64(6), int64(1), object(6)
memory usage: 

In [ ]:
# save data first, this is from API call.
# cleaned_merged_phone_df.to_csv("dirty_merged_data.csv",index=False)

#### e. Data Cleaning Implementation (Part 3)

In [2]:
# read data for next pre-processing
cleaned_merged_phone_df2 = pd.read_csv("dirty_merged_data.csv")

In [3]:
cleaned_merged_phone_df2.brand_name.value_counts()

brand_name
Brand Name: HP                                       38
Brand Name: Samsung                                  22
Brand Name: Vivo                                     21
Brand Name: Xiaomi                                   21
Brand Name: Unknown                                  20
Brand Name: Hp                                       18
Brand Name: VIVO                                     16
Brand Name: Infinix                                  12
Brand Name: OPPO                                     10
Brand Name: Nokia                                     8
Brand Name: Oppo                                      7
Brand Name: TECNO                                     5
Brand Name: REALME                                    4
Brand Name: Tecno                                     4
Brand Name: SAMSUNG                                   4
Brand Name: Sharp                                     3
Brand Name: Realme                                    2
Brand Name: IQOO                     

#### Insight:
1. We need to remove "Brand Name:"
2.  Unify the brand names:
- realmi, Realme, REALME => Realme
- Pova, Tecno, TECNO => TECNO
- iPhone, IPHON, Apple => Apple
- SAMSUNG,Samsung, GALAXY, Galaxy => Samsung
- Reno, OPPO,Oppo => OPPO
- Huawei, HUAWEI => Huawei
- Vivo, VIVO => VIVO
- Zte => ZTE
- Mi, XIAOMI, Redmi, REDMI, Xiaomi => Xiaomi
- nokia, NOKIA => Nokia
- HANDPHONE GP => GP
- ADVAN => Advan
- Infinix, INFINIX => Infinix
3. investigate non-representing brand names:
- HP, Ponsel, Hp (Hewlett-Packard', Unknown, hp, The brand name extracted from the product name "V'.


In [4]:
# Define the mapping of old brand names to new unified brand names
brand_mapping = {
    "realmi": "Realme",
    "REALME": "Realme",
    "Pova": "TECNO",
    "Tecno": "TECNO",
    "iPhone": "Apple",
    "IPHON": "Apple",
    "SAMSUNG": "Samsung",
    "GALAXY": "Samsung",
    "Galaxy": "Samsung",
    "Reno": "OPPO",
    "Oppo": "OPPO",
    "HUAWEI": "Huawei",
    "Vivo": "VIVO",
    "Zte": "ZTE",
    "Mi": "Xiaomi",
    "XIAOMI": "Xiaomi",
    "Redmi": "Xiaomi",
    "REDMI": "Xiaomi",
    "nokia": "Nokia",
    "NOKIA": "Nokia",
    "HANDPHONE GP": "GP",
    "ADVAN": "Advan",
    "INFINIX": "Infinix" 
}

In [5]:
# remove "Brand name: "
cleaned_merged_phone_df2['brand_name'] = cleaned_merged_phone_df2['brand_name'].str.replace("Brand Name: ", "", regex=True)
# Use map to unify brand names
cleaned_merged_phone_df2['brand_name'] = cleaned_merged_phone_df2['brand_name'].map(brand_mapping).fillna(cleaned_merged_phone_df2['brand_name'])
# Display the updated DataFrame
cleaned_merged_phone_df2.brand_name.unique()

array(['TECNO', 'VIVO', 'HP', 'ITEL', 'Apple', 'Hp (Hewlett-Packard',
       'OPPO', 'Unknown', 'Hp', 'Samsung', 'Sharp', 'Xiaomi', 'Infinix',
       'Prince', 'Realmi', 'Nokia', 'HP (Hewlett-Packard', 'Huawei',
       'Realme', 'Advan', 'Ponsel', 'IQOO',
       'The brand name extracted from the product name "V', 'GP', 'VOVE',
       'Baru', 'Hush Puppies', 'hp', 'LENOVO', 'RESMI', 'ASUS',
       'EVERCOSS', 'ZTE'], dtype=object)

In [6]:
print(cleaned_merged_phone_df2.brand_name.value_counts())

brand_name
HP                                                   38
VIVO                                                 37
Samsung                                              29
Xiaomi                                               27
Unknown                                              20
Hp                                                   18
OPPO                                                 18
Infinix                                              13
TECNO                                                10
Nokia                                                 9
Realme                                                7
Apple                                                 4
Huawei                                                4
Sharp                                                 3
ITEL                                                  2
hp                                                    2
IQOO                                                  2
EVERCOSS                             

In [7]:
# checking what's with HP(Hewlett-Packard)
# Create a boolean mask for the desired brand names
hewlet_mask = (cleaned_merged_phone_df2['brand_name'] == "Hp (Hewlett-Packard") | (cleaned_merged_phone_df2['brand_name'] == "HP (Hewlett-Packard")
# Apply the mask to the DataFrame
hewlet_filtered_df = cleaned_merged_phone_df2[hewlet_mask]
# hewlet_filtered_df

##### Insight:
- change Hp (Hewlett-Packard -> OPPO since it's a Reno Phone
- change HP(Hewlett-Packard -> Advan since it's an Advan Phone

In [8]:
# checking what's with Ponsel, The brand name extracted from the product name "V , hp, Baru,RESMI
# Create a boolean mask for the desired brand names
filtered_df_2 = cleaned_merged_phone_df2[cleaned_merged_phone_df2['brand_name'].isin(["Ponsel",""" The brand name extracted from the product name "V""", "hp",'Baru','RESMI'])]
# Apply the mask to the DataFrame
# filtered_df_2

##### Insight:
- Ponsel => Samsung (Galaxy A16)
- Baru => Infinix (Hot30)
- hp => Samsung (S23), Apple (i14)
- RESMI => Xiaomi (NOTE 11)


In [9]:
# checking what's with Ponsel, The brand name extracted from the product name "V , hp, Baru,RESMI
# Create a boolean mask for the desired brand names
filtered_df_3 = cleaned_merged_phone_df2[cleaned_merged_phone_df2['brand_name'].isin(["HP"])]
# Apply the mask to the DataFrame
# filtered_df_3

##### Insight:
- A96, A53s => OPPO
- Note 12,M4 Pro, NOTE 12,Note 11, XlAOMl => Xiaomi
- I14, IP14 => Apple
- Vivo, Y17, V27e, V29e, V27eV29e, Y22, Y21A, VIV0=> VIVO
- zee => ZEE 
- S23,A23,A52, A13, Galaxy, A32, A16,S8 => Samsung
- Fujitsu => Fujitsu
- C55, C35 => Realme

#### **Insight Summary on Cleaning**:
- A96, A53s, Reno9 => OPPO
- Baru => Infinix
- Note 12,M4 Pro, NOTE 12,Note 11,NOTE 11, XlAOMl => Xiaomi
- I14, IP14,i14 => Apple
- Vivo, Y17, V27e, V29e, V27eV29e, Y22, Y21A, VIV0=> VIVO
- zee => ZEE 
- S23,A23,A52, A13, Galaxy, A32, A16,S8 => Samsung
- Fujitsu => Fujitsu
- C55, C35 => Realme
- HP(Hewlett-Packard -> Advan

In [10]:
# mapping function
def replace_brand_name(df, name_col, brand_col):
    # Define the keyword to new_brand_name mapping
    keyword_mapping = {
        "A96": "OPPO",
        "A53s": "OPPO",
        "Reno9": "OPPO",
        "oppo": "OPPO",
        "reno 9": "OPPO",
        "Xiaomi": "Xiaomi",
        "redmi": "Xiaomi",
        "Note 12": "Xiaomi",
        "M4 Pro": "Xiaomi",
        "NOTE 12": "Xiaomi",
        "Note 11": "Xiaomi",
        "NOTE 11": "Xiaomi",
        "XlAOMl": "Xiaomi",
        "sku : 3131/1533839326389420032/55":"Xiaomi",
        "i12": "Apple",
        "I14": "Apple",
        "IP14": "Apple",
        "i14": "Apple",
        "i15": "Apple",
        "Vivo": "VIVO",
        "Y17": "VIVO",
        "V27e": "VIVO",
        "v29": "VIVO",
        "V29e": "VIVO",
        "V27eV29e": "VIVO",
        "Y22": "VIVO",
        "Y21A": "VIVO",
        "VIV0": "VIVO",
        "t1 pro": "VIVO",
        "y21s": "VIVO",
        "zee": "ZEE",
        "Galaxy": "Samsung",
        "S23": "Samsung",
        "S22": "Samsung",
        "A23": "Samsung",
        "A52": "Samsung",
        "A13": "Samsung",
        "10A": "Samsung",
        "A32": "Samsung",
        "A16": "Samsung",
        "S8": "Samsung",
        "Fujitsu": "Fujitsu",
        "C55": "Realme",
        "C35": "Realme",
        "Realmi":"Realme",
        "HOT 30": "Infinix",
        "HOT30": "Infinix",
        "LUNA": "Luna",
        "prince":"Prince",
        "p40pro": "Huawei",
        "advan": "Advan"
        
            }

    # Iterate over the mapping and update the brand_name column
    for keyword, new_brand in keyword_mapping.items():
        df.loc[df[name_col].str.contains(keyword, case=False, na=False, regex=False), brand_col] = new_brand

    return df

In [11]:
# Apply the function
cleaned_merged_phone_df2 = replace_brand_name(cleaned_merged_phone_df2, name_col='name', brand_col='brand_name')
# Display the updated DataFrame
print(cleaned_merged_phone_df2.brand_name.unique())
print("\n")
print(cleaned_merged_phone_df2.brand_name.value_counts())

['TECNO' 'VIVO' 'Xiaomi' 'ITEL' 'Apple' 'OPPO' 'Samsung' 'Sharp' 'Infinix'
 'Huawei' 'Prince' 'Realme' 'Nokia' 'HP' 'HP (Hewlett-Packard' 'Unknown'
 'Advan' 'IQOO' 'Fujitsu' 'Luna' 'GP' 'VOVE' 'Hush Puppies' 'LENOVO'
 'ASUS' 'ZEE' 'EVERCOSS' 'Hp' 'ZTE']


brand_name
VIVO                   60
Samsung                46
Xiaomi                 37
OPPO                   20
Infinix                17
Realme                 14
Apple                  12
TECNO                  10
Nokia                   9
Huawei                  5
HP                      4
Unknown                 4
Sharp                   3
IQOO                    2
ITEL                    2
ASUS                    1
ZEE                     1
Hp                      1
EVERCOSS                1
Hush Puppies            1
LENOVO                  1
HP (Hewlett-Packard     1
VOVE                    1
GP                      1
Luna                    1
Fujitsu                 1
Advan                   1
Prince                  1
ZTE  

In [12]:
# checking what's with Ponsel, The brand name extracted from the product name "V , hp, Baru,RESMI
# Create a boolean mask for the desired brand names
filtered_df_4 = cleaned_merged_phone_df2[cleaned_merged_phone_df2['brand_name'].isin(["HP","Hp",'Unknown'])]
# Apply the mask to the DataFrame
# filtered_df_4

In [13]:
# Set the maximum column width to display full content
pd.set_option('display.max_colwidth', None)
# Now print the column
print(filtered_df_4.product_description)

37                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

#### Insight:
- we need to check on product_description for finding the brand.
- keyword found: prince, oppo, v29,sku : 3131/1533839326389420032/55 (Realme),p40pro (Huawei).

In [14]:
# Apply the function
cleaned_merged_phone_df2 = replace_brand_name(cleaned_merged_phone_df2, name_col='product_description', brand_col='brand_name')
# Display the updated DataFrame
print(cleaned_merged_phone_df2.brand_name.unique())
print("\n")
print(cleaned_merged_phone_df2.brand_name.value_counts())

['TECNO' 'VIVO' 'Xiaomi' 'ITEL' 'Apple' 'OPPO' 'Samsung' 'Sharp' 'Infinix'
 'Huawei' 'Prince' 'Nokia' 'HP' 'Luna' 'Realme' 'Advan' 'Unknown' 'IQOO'
 'Fujitsu' 'GP' 'VOVE' 'Hush Puppies' 'LENOVO' 'ASUS' 'ZEE' 'EVERCOSS'
 'ZTE']


brand_name
VIVO            63
Samsung         46
Xiaomi          38
OPPO            22
Infinix         17
Apple           12
Realme          11
TECNO           10
Nokia            9
Huawei           5
Fujitsu          2
IQOO             2
Unknown          2
Advan            2
Luna             2
HP               2
Prince           2
Sharp            2
ITEL             2
GP               1
VOVE             1
Hush Puppies     1
LENOVO           1
ASUS             1
ZEE              1
EVERCOSS         1
ZTE              1
Name: count, dtype: int64


In [15]:
# Create a boolean mask for the desired brand names
filtered_df_5 = cleaned_merged_phone_df2[cleaned_merged_phone_df2['brand_name'].isin(["HP",'Unknown'])]
# Apply the mask to the DataFrame
# filtered_df_5

#### Insight:
1. after confirming products from their links, the product names:
- HP Android Murah Ram 8GB/256GB Garansi 1 Tahun => VIVO
- handphone android murah => Maxtron
- Hp murah 4G -> Unknown

In [16]:
# Replace brand_name based on specific conditions
cleaned_merged_phone_df2.loc[
    (cleaned_merged_phone_df2['name'] == "HP Android Murah Ram 8GB/256GB Garansi 1 Tahun") &
    (cleaned_merged_phone_df2['store_name'] == "Trendy Store 44") &
    (cleaned_merged_phone_df2['price'] == 1119000.0),
    'brand_name'
] = "VIVO"

cleaned_merged_phone_df2.loc[
    (cleaned_merged_phone_df2['name'] == "handphone android murah") &
    (cleaned_merged_phone_df2['store_name'] == "Sehun Ponsel") &
    (cleaned_merged_phone_df2['price'] == 1350000.0),
    'brand_name'
] = "Maxtron"

cleaned_merged_phone_df2.loc[
    (cleaned_merged_phone_df2['name'] == "Hp murah 4G") &
    (cleaned_merged_phone_df2['store_name'] == "Aaurelia9910") &
    (cleaned_merged_phone_df2['price'] == 2000000.0),
    'brand_name'
] = "Unknown"

# Display the updated DataFrame
print(cleaned_merged_phone_df2.brand_name.unique())

['TECNO' 'VIVO' 'Xiaomi' 'ITEL' 'Apple' 'OPPO' 'Samsung' 'Sharp' 'Infinix'
 'Huawei' 'Prince' 'Nokia' 'Unknown' 'Luna' 'Realme' 'Advan' 'Maxtron'
 'IQOO' 'Fujitsu' 'GP' 'VOVE' 'Hush Puppies' 'LENOVO' 'ASUS' 'ZEE'
 'EVERCOSS' 'ZTE']


In [17]:
# Create a boolean mask for the desired brand names
filtered_df_6 = cleaned_merged_phone_df2[cleaned_merged_phone_df2['brand_name'].isin(["Hush Puppies",'Unknown'])]

# Apply the mask to the DataFrame
filtered_df_6

,link,name,price,rating,total_sold,store_name,store_location,product_description,customer_satisfaction,IsRated,total_rating,num_reviews,review_rating_ratio,brand_name
37,https://www.tokopedia.com/archive-aaurelia9910/hp-murah-4g?extParam=fcity%3D174%2C175%2C176%2C177%2C178%2C179%23144%2C146%2C150%2C151%2C167%2C168%2C171%2C174%2C175%2C176%2C177%2C178%2C179%2C463%23165%2346%23252%26ivf%3Dfalse%26keyword%3Dhp+murah%26search_id%3D202503010158437659E76192BAE0069WUB%26src%3Dsearch,Hp murah 4G,2000000.0,5.0,3,Aaurelia9910,Medan,"ram 2 kamera depan 13mp baterai 2500 mah layar hd+ 5,7inc",100.0,True,2.0,0.0,0.0,Unknown
144,https://www.tokopedia.com/archive-akmalcel/handphone-murah?extParam=fcity%3D174%2C175%2C176%2C177%2C178%2C179%23144%2C146%2C150%2C151%2C167%2C168%2C171%2C174%2C175%2C176%2C177%2C178%2C179%2C463%23165%2346%23252%26ivf%3Dfalse%26keyword%3Dhp+murah%26search_id%3D202503010158437659E76192BAE0069WUB%26src%3Dsearch,handphone murah,1450000.0,5.0,8,Akmal Cel,Medan,baru,100.0,True,8.0,0.0,0.0,Unknown
150,https://www.tokopedia.com/antao-1/dijual-hush-puppies-men-s-watches-hp-3852m-2519-murah?extParam=fcity%3D174%2C175%2C176%2C177%2C178%2C179%23144%2C146%2C150%2C151%2C167%2C168%2C171%2C174%2C175%2C176%2C177%2C178%2C179%2C463%23165%2346%23252%26ivf%3Dfalse%26keyword%3Dhp+murah%26search_id%3D202503010158437659E76192BAE0069WUB%26src%3Dsearch,Dijual Hush Puppies Men's Watches HP 3852M.2519 Murah,1906700.0,2.5,0,Antaolshp,Jakarta Barat,"hush puppies men #39;s watches hp 3852m.2519 leather strap, jam tangan analog yang didesain dengan: - movement : analog - case : stainless steel (d = 41 mm) - band : leather (w = 24 mm) - lens dial : mineral crystal - water resistant : 5 atm - garansi : 2 (dua) tahun internasional - kelengkapan paket : 1x watch, 1x kartu garansi, 1x box sku : 3131/1404080896830029824/55",0.0,False,0.0,0.0,0.0,Hush Puppies


#### f. Finalized Clean Data for Visualization

In [18]:
def unify_city_name(city_name):
    # Define the unification rules
    if 'Jakarta' in city_name:
        return 'Jakarta'
    elif 'Tangerang' in city_name:
        return 'Tangerang'
    elif 'Bekasi' in city_name:
        return 'Bekasi'
    elif city_name == 'Kab. Bogor':
        return 'Bogor'
    else:
        return city_name  # Return the original name if no rule applies

In [ ]:
# data filtering
final_filter = (cleaned_merged_phone_df2['brand_name'] == "Hush Puppies")

# get new data version
cleaned_merged_phone_df3 = cleaned_merged_phone_df2[~final_filter]
# Apply the function to the 'city' column
cleaned_merged_phone_df3.loc[:, 'store_location'] = cleaned_merged_phone_df3['store_location'].apply(unify_city_name)
cleaned_merged_phone_df3.loc[:,'product_description'] =  cleaned_merged_phone_df3['product_description'].fillna("No product description available.")

# for brands that have < 5

# sort based on total sold
sorted_cleaned_merged_phone_df3 = cleaned_merged_phone_df3.sort_values(by='total_sold',ascending=False)
# show total data
sorted_cleaned_merged_phone_df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 258 entries, 39 to 258
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   link                   258 non-null    object 
 1   name                   258 non-null    object 
 2   price                  258 non-null    float64
 3   rating                 258 non-null    float64
 4   total_sold             258 non-null    int64  
 5   store_name             258 non-null    object 
 6   store_location         258 non-null    object 
 7   product_description    258 non-null    object 
 8   customer_satisfaction  258 non-null    float64
 9   IsRated                258 non-null    bool   
 10  total_rating           258 non-null    float64
 11  num_reviews            258 non-null    float64
 12  review_rating_ratio    258 non-null    float64
 13  brand_name             258 non-null    object 
dtypes: bool(1), float64(6), int64(1), object(6)
memory usage: 28.5

In [207]:
# clean data
pd.set_option('display.width', 1000)  # Adjust the value as needed
sorted_cleaned_merged_phone_df3.head()

,link,name,price,rating,total_sold,store_name,store_location,product_description,customer_satisfaction,IsRated,total_rating,num_reviews,review_rating_ratio,brand_name
39,https://www.tokopedia.com/samsung/samsung-galaxy-a05s-6-128gb-silver-e3ea5?extParam=ivf%3Dfalse%26keyword%3Dhp+murah&src=topads,Samsung Galaxy A05s 6/128GB,1999000.0,2.5,8000,Samsung Official Store,Jakarta,"""awesome offers! - free travel adapter 25w senilai rp.299.000** - free youtube premium 2 bulan* *syarat & ketentuan berlaku. klaim melalui aplikasi samsung gift indonesia. **selama persediaan masih ada"" spesifikasi : - processor : snapdragon 680 - size : 6.7"" - technology : pls lcd, 90 hz - resolution : fhd+ - rear camera resolution : 50mp + 2mp + 2mp - main camera auto focus : yes - rear camera - ois : no - rear camera - zoom : digital zoom up to 10x - front camera resolution : 13.0 mp - front camera auto focus : no - video resolution : fhd (1920 x 1080)@30fps - ram : 6gb - rom : 128gb - sim tray : dual-sim - network type :4g - usb interface : data cable c to c - usb version : usb 2.0 - earjack : yes - bluetooth version : bluetooth v5.1 - nfc : no - battery capacity : 5000mah 25w fast charging support - side fingerprint : yes",99.0,False,4155.0,1475.0,0.354994,Samsung
41,https://www.tokopedia.com/samsung/samsung-galaxy-a25-5g-8-256gb-blue-0a50b?extParam=ivf%3Dfalse%26keyword%3Dhp+murah&src=topads,Samsung Galaxy A25 5G 8/256GB,4049000.0,2.5,750,Samsung Official Store,Jakarta,"spesifikasi -processor : octa-core -size : 6.5"" -technology : super amoled 120hz -resolution : fhd+ -rear camera resolution : 50.0 mp + 8.0 mp + 2.0 mp -main camera auto focus : yes -rear camera - ois : yes -rear camera - zoom : digital zoom up to 10x -front camera resolution : 13.0 mp -front camera auto focus : no ""-video resolution : uhd 4k (3840 x 2160)@30fps slow motion: 480fps @hd,240fps @hd"" -ram : 8gb -rom : 256gb -sim tray : dual-sim -network type : 5g -usb interface : usb type-c -usb version : usb 2.1 -earjack : 3.5mm stereo -bluetooth version : bluetooth v5.3 -nfc : yes -battery capacity : 5000 mah -25w fast charging",98.0,False,438.0,169.0,0.385845,Samsung
19,https://www.tokopedia.com/distriponsel/xiaomi-redmi-14c-8-256-gb-6-128-gb-redmi-14-c-garansi-resmi-not-12-c-13-c-14c-6-128-black-non-bundle-f53bb?extParam=ivf%3Dtrue%26keyword%3Dhp+murah&src=topads,Xiaomi Redmi 14C 8/256 GB 6/128 GB Redmi 14 C Garansi Resmi Not 12 C 13 C,1385000.0,2.5,500,Distributor Ponsel,Jakarta,"untuk produk xiaomi, vivo, realme, oppo, samsung, infinix kami unseal semua yaa, hanya untuk aktivasi, selebihnya dijamin tidak ada yang ditukar/ganti tersedia juga aksesoris kompatible dengan produk ini : anti gores: https://www.tokopedia.com/distriponsel/anti-gores-jelly-tempered-glass-hydrogel-xiaomi-samsung-vivo-oppo-real tws: https://www.tokopedia.com/distriponsel/pioneer-earphone-series-headset-hp-murah-pioneer powerbank: https://www.tokopedia.com/distriponsel/realme-powerbank-power-bank-2i-10000mah-quick-charge-dual-output packing yang lebih aman (opsional, untuk standar packing kami sudah memakai bubble wrap) : ekstra bubble: https://www.tokopedia.com/distriponsel/ekstra-bubble-wrap-untuk-packing packing kayu: https://www.tokopedia.com/distriponsel/packing-kayu-untuk-handphone review customer distributor ponsel : https://www.tokopedia.com/distriponsel/review mengenai varian : non bundle / variant warna = tidak dapat bonus yang seperti digambar bundle headset = dapat bonus headset bundle powerbank = dapat bonus realme powerbank bundle hydrogel = dapat bonus anti gores hydrogel bundle lengkap = dapat semua bonus yang disebutkan diatas untuk varian promo barang sama, harga lebih murah karena varian tersebut dapat diskon dari tokopedia, untuk warna yang ready bisa cek divarian dan wajib cantumkan warna dicatatan ketika order. jika tidak, kami kirim sesuai stock yang tersedia dan tidak bisa komplain warna. *dengan syarat & ketentuan. tersedia 2 variant : xiaomi redmi 14c 6/128gb 8/256gb black, blue,

In [ ]:
# save data
# sorted_cleaned_merged_phone_df3.to_csv("final_cleaned_data.csv",index=False)